# Interogate Placebo Literature with RAG, and Text to Speech with BARK.

# Install Dependencies

In [ ]:

"""
Make sure jupyter can use the kernel from the correct virtual environment:
https://saturncloud.io/blog/running-jupyter-notebook-in-a-virtual-environment-installed-scikitlearn-module-not-available/
"""
# install dependencies
! pip install langchain openai chromadb langchainhub
! pip install pydantic -U
! pip install tiktoken
! pip install pypdf
! pip install -U langchain-community
! pip install git+https://github.com/suno-ai/bark.git
! pip install git+https://github.com/huggingface/transformers.git



# Import Requirements

In [1]:
import os
import openai
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OpenAIEmbeddings
from langchain import hub
from langchain_community.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from transformers import AutoProcessor, BarkModel
from IPython.display import Audio

# Set Paths and Key

In [46]:
pdf_folder = "/Users/matt/Folders/Programming/Packt_GenerativeAIwithPythonAndTensorflow2/LangChain/Pubmed-Batch-Download/placebo effect_300/working_pdfs"
db_path = os.path.join(pdf_folder,"chroma_db")
db_exists = os.path.exists(db_path)

openai.api_key = os.environ["OPENAI_API_KEY"] # this is set in the source ~/.zshrc
working_pdf_folder = os.path.join(pdf_folder, 'working_pdfs')


# Create or Load Vector Database 

In [47]:
if not db_exists:
    # using a full directory of pdf's
    loader2 = PyPDFDirectoryLoader(working_pdf_folder)
    # Split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
    splits = text_splitter.split_documents(loader2.load())
    # save to disk
    vectorstore = Chroma.from_documents(documents=splits,
                                        embedding=OpenAIEmbeddings(),
                                        persist_directory=os.path.join(pdf_folder,"chroma_db"))
else:
    # load from disk
    vectorstore = Chroma(persist_directory=os.path.join(pdf_folder,"chroma_db"), embedding_function=OpenAIEmbeddings())

# Define LLM and Rag Prompt

In [48]:
retriever = vectorstore.as_retriever()
rag_prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
)


# Define Query, Retrieve Response

In [52]:
#query = "What is the mechanism of action of placebos?"
query = "What are the factors that influence the strength of the effect from a deceptively perscribed placebo?"
response = rag_chain.invoke(query)
print(response.content)
response_part = response.content.split(',')[0]

The strength of the placebo effect can be influenced by disease-specific and disease-unspecific factors, with one study suggesting that low symptom severity at baseline is predictive of a higher placebo response. However, this finding was not confirmed in all studies. Additionally, the perception of placebos can be affected by whether they are prescribed openly or deceptively, with deceptive placebos potentially being seen as more powerful.


# Define BARK Model

In [ ]:
processor = AutoProcessor.from_pretrained("suno/bark")
model = BarkModel.from_pretrained("suno/bark")
voice_preset = "v2/en_speaker_6"

# Generate Audio Response

In [ ]:
inputs = processor(response_part, voice_preset=voice_preset)
audio_array = model.generate(**inputs)
audio_array = audio_array.cpu().numpy().squeeze()

# Play Audio Response

In [45]:

Audio(audio_array, rate=model.generation_config.sample_rate)

# Find Relevant Literature Sources Relevant to Query

In [53]:
def get_refs(query):
    docs = vectorstore.similarity_search(query)
    for ans in range(len(docs)):
        print("\n=====================================\n")
        print("Source Document:")
        print(docs[ans].metadata['source'].split('/')[-1])
        print("\n")
        print(f"Page Number:")
        print(docs[ans].metadata['page'])
        print("\n")
        print("Relevant Content:")
        print(docs[ans].page_content.replace("\n"," "))
        print("\n")

get_refs(query)



Source Document:
The 'placebo effect' in the conservative treatment of plantar fasciitis: a systematic review and met.pdf


Page Number:
7


Relevant Content:
investigated other possible factors influencing the placebo  effect. Weimer et  al.  (71) suggested that both disease- specific as well as disease-unspecific factors can influence  the response to placebo treatment in RCTs. In particular,  the most predictive individual factor for a higher placebo  response was a low symptom severity at baseline ( 72). This  finding was not confirmed in the present study, where,  according to the meta-regression performed, factors




Source Document:
Placebo and nocebo effects and mechanisms associated with pharmacological interventions: an umbrella.pdf


Page Number:
0


Relevant Content:
been shown that many biological mechanisms triggered by placebos and nocebos resemble those modulated by drugs, suggesting a possible interaction between psychological factors and drug action. 6 In 2018, a c